In [1]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt
import locale
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time


In [2]:
dbname = mongo_utils.get_database()
url=''

/home/kj/conda/envs/lab/lib/python3.6/site-packages/pymongo/pyopenssl_context.py:26: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.x509 import load_der_x509_certificate as _load_der_x509_certificate


In [3]:
url='https://novini.bg/latest-news'

In [4]:
collection_name = dbname["novinibg_scrape_1"]
manin_page = requests.get(url)
man_page_data = manin_page.text
main_page = BeautifulSoup(man_page_data, 'html.parser')
for scr_url in main_page.find_all('a', class_="pagination__link -theme-bg-block -theme-border-pagination", href=True): 
    #print(scr_url["href"])
    manin_page = requests.get(scr_url["href"])
    man_page_data = manin_page.text
    main_page = BeautifulSoup(man_page_data, 'html.parser')
    for scr_url2 in main_page.find_all('article', class_="g-grid__item js-content"):   
        for final_url in scr_url2.find_all('a', href=True):
            nlink=final_url["href"]
            #print(nlink)
            for scr_url in main_page: #.find_all('article', class_="g-grid__item js-content", href=True):
              try:        
                #print(nlink)
                if not collection_name.count_documents({"news_link" : nlink}, limit = 1):
                    manin_page = requests.get(nlink)
                    man_page_data = manin_page.text
                    main_page = BeautifulSoup(man_page_data, 'html.parser')
                    news_title=(main_page.find('title').get_text())
                    for scr_entr in main_page.find_all('section', class_="openArticle__content --spaced-bottom"):
                        test_lines=scr_entr.find_all('p')
                        art_text=''
                        for line in test_lines:
                            plain_text=line.get_text().replace('  ','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
                            art_text += plain_text
                        #print(art_text)
                    text=art_text
                    dta=main_page.find('p', class_="openArticle__date").get_text().replace('  ','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
                    locale.setlocale(locale.LC_ALL, 'bg_BG.UTF-8')
                    date_posted=dt.strptime(dta, '%d %b %Y %H:%M').timestamp()
                    news_payload = {
                    "date_posted" : date_posted,
                    "news_link" : nlink,
                    "news_text" : text,
                    "news_title" : news_title
                    }
                    print(news_payload)
                    collection_name.insert_many([news_payload])
              except:
                pass

{'date_posted': 1655744160.0, 'news_link': 'https://novini.bg/bylgariya/politika/724443', 'news_text': '„Има толкова много хора, които искат да има България без задкулисие. Промяната зависи от всички нас". Това заяви министър-председателят Кирил ПетковKирил Петков Петков е роден през 1980 г. Той е един от основатели на Центъра за Икономически по време на демонстрацията пред НС в подкрепа на правителството, предаде репортер на Novini.bg."Непрекъснато говорим с различни депутати. Тук е въпросът на страх и отговорност. Надявам се много хора да направят тази стъпка, която изглежда страшна, но е отговорна", каза Петков. НА ЖИВО: Граждани се събраха пред НС в защита на правителствотоПо повод планираните протести утре, Петков заяви:"Надявам се, че няма да има никакви проблеми, г-н Рашков гарантира за това. Така че, ще се опитаме да има само мирни протести и хората, които ни подкрепят, наистина да изявят, че тази промяна за всички нас", увери премиерът.Кирил ПетковKирил Петков Петков е роден п